In [1]:
#Import dependencies
import pandas as pd
import json
from pprint import pprint as pp
import pymongo
from pymongo import MongoClient
from pandas.io.json import json_normalize

In [2]:
#Set up mongodb connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.pro3news
collection = db.shares

In [3]:
#Loop through Newspaper json files
newspapers = ['Breitbart','Business_Insider','Fox News','Financial Times','MarketWatch','NYTimes','Reuters','The Guardian','WSJ']
times = ['1_1','1_2','1_3','1_4','2_1','2_2','2_3','2_4','3_1','3_2','3_3','3_4','4_1','4_2']

for paper in newspapers:
    for time in times:
        with open(paper+'/'+time+'.json') as json_file:
            news_data = json.load(json_file)

In [4]:
#Normalize the json data
for data in news_data['articles']['results']:
    total_data_df = pd.DataFrame.from_dict(json_normalize(data), orient='columns')


In [5]:
#Pull the News Share columns and rename them
news_shares_df = total_data_df.loc[:,['source.title','date','title','shares.facebook']]
news_shares_df.rename(columns={'source.title':'newspaper',
                                'shares.facebook':'shares'},
                     inplace=True)


In [6]:
#Change dataframe back to json
news_shares_dict = {}
news_shares_dict = news_shares_df.to_dict('records')

#Add the data to mongodb
collection.insert_many(news_shares_dict)



[{'newspaper': 'The Wall Street Journal',
  'date': '2020-04-06',
  'title': "Need a Small Business Loan? Here's What You Need to Know",
  'shares': 1298,
  '_id': ObjectId('5e9c8524e6433dc081507a20')}]